In [2]:
import gym
from gym import wrappers
import numpy as np

In [16]:
for i in gym.envs.registry:
    #print(i)

SyntaxError: incomplete input (159489049.py, line 2)

In [125]:
class ANN:
    def __init__(self, input_size, hidden_size, output_size):
        self.learning_rate = 0.005
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights
        self.W1 = np.random.randn(hidden_size, input_size) * np.sqrt(1/input_size)
        self.b1 = np.zeros((hidden_size, ))
        self.W2 = np.random.randn(output_size, hidden_size) * np.sqrt(1/output_size)
        self.b2 = np.zeros((output_size, ))
    
    def forward(self, observation):
        print("Shape of b1:", self.b1.shape)
        self.hidden_in = np.dot(self.W1, observation) + self.b1

        self.hidden_out = self.relu(self.hidden_in)
        self.output_in = np.dot(self.W2, self.hidden_out) + self.b2        
        self.output_out = self.sigmoid(self.output_in)
        print(self.output_out.shape)
        return self.output_out
    
    def backprop(self, observation, Error):
        # Backpropagation
        reluOutput = self.sigmoid_backward(self.output_in)
        print("Shape of ReluOutput: ", reluOutput.shape)

        delta2 = reluOutput * Error 
        print("Shape of delta 2:", delta2.shape)

        dw2 = np.outer(delta2, self.hidden_out)
        print("Shape of dw2:", dw2.shape)

        db2 = delta2
        print("shape of db2:", db2.shape)

        # Backpropagation: hidden layer -> input layer
        reluInput = self.relu_backward(self.hidden_in)
        print("Shape of ReluInput: ", reluInput.shape)

        delta1 = reluInput * np.dot(self.W2.T, delta2)
        print("Shape of delta 1:", delta1.shape)

        dw1 = np.outer(delta1, observation)
        print("Shape of dw1:", dw1.shape)

        db1 = delta1
        print("Shape of db1:", db1.shape)
       
        # Update Rules
        self.W1 += self.learning_rate * dw1
        self.W2 += self.learning_rate * dw2
        self.b1 += self.learning_rate * db1
        self.b2 += self.learning_rate * db2  

    def softmax(self, Z):
        e_Z = np.exp(Z - np.max(Z, axis=0))
        return e_Z / np.sum(e_Z, axis=0)
    
    def sigmoid(self, Z):
        return 1/(1+np.exp(-Z))

    def relu(self, Z):
        return np.maximum(0,Z)

    def sigmoid_backward(self, layer):
        sig = self.sigmoid(layer)
        return sig * (1 - sig)

    def relu_backward(self, layer):
        return np.where(layer > 0, 1, 0)


In [132]:
epsilon_0 = 0.6 
beta = 0.00005
gamma = 0.85  
eta = 0.005 

env = gym.make('CartPole-v1', render_mode="human")
observation = env.reset()

s_size = env.observation_space.shape[0]
a_size = env.action_space.n
hidden_size = 8

net = ANN(s_size, hidden_size, a_size)
print(s_size, hidden_size, a_size)
for i in range(1000):
    done = False
    observation = env.reset()
    observation = observation[0]
    env.render()

    while not done:

        # Forward prop
        Q_values = net.forward(observation)
        
        # Policy Decision
        Q_current = np.max(Q_values)
        action = np.argmax(Q_values)
        # Next step
        observation_next, reward, done, info, _ = env.step(action)
        print(observation)
        print(observation_next)
        if not done: 
            print("if")
            Error = reward - Q_current
            net.backprop(observation[0], Error)
        else:
            print("else")
            Q_values_next = net.forward(observation_next)
            Q_max = np.max(Q_values_next)
            Q_target = reward + gamma * Q_max 
            Error = Q_target - Q_current 
            net.backprop(observation, Error)
        
        observation = observation_next

        if done:
            break
env.close()

4 8 2
Shape of b1: (8,)
(2,)
[-0.01552711  0.03339009 -0.02227757 -0.04552679]
[-0.0148593   0.22882429 -0.0231881  -0.3451544 ]
if
Shape of ReluOutput:  (2,)
Shape of delta 2: (2,)
Shape of dw2: (2, 8)
shape of db2: (2,)
Shape of ReluInput:  (8,)
Shape of delta 1: (8,)
Shape of dw1: (8, 1)
Shape of db1: (8,)
Shape of b1: (8,)
(2,)
[-0.0148593   0.22882429 -0.0231881  -0.3451544 ]
[-0.01028282  0.03403973 -0.03009119 -0.05987282]
if
Shape of ReluOutput:  (2,)
Shape of delta 2: (2,)
Shape of dw2: (2, 8)
shape of db2: (2,)
Shape of ReluInput:  (8,)
Shape of delta 1: (8,)
Shape of dw1: (8, 1)
Shape of db1: (8,)
Shape of b1: (8,)
(2,)
[-0.01028282  0.03403973 -0.03009119 -0.05987282]
[-0.00960202  0.22957993 -0.03128865 -0.3618958 ]
if
Shape of ReluOutput:  (2,)
Shape of delta 2: (2,)
Shape of dw2: (2, 8)
shape of db2: (2,)
Shape of ReluInput:  (8,)
Shape of delta 1: (8,)
Shape of dw1: (8, 1)
Shape of db1: (8,)
Shape of b1: (8,)
(2,)
[-0.00960202  0.22957993 -0.03128865 -0.3618958 ]
[-0.00